<a href="https://colab.research.google.com/github/Ahmadfikriand/R-and-Python-Code/blob/master/COMP8044041_TP2_2402313221_Ahmad_Fikri_Andrismono.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**EVALUATING OBJECT DETECTIONS WITH FIFTYONE FROM COCO DATASETS**


Dalam code kali ini, akan memanfaatkan model Faster R-CNN yang diberikan oleh PyTorch. Pastikan memasang torch dan torchvision.

In [1]:
!pip install fiftyone
!pip install torch torchvision

Berikut ini adalah potongan kode untuk mengunduh dan memuat model yang telah dilatih dari web.

In [2]:
import torch
import torchvision

# Run the model on GPU if it is available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Load a pre-trained Faster R-CNN model
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
model.to(device)
model.eval()

print("Model ready")

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Model ready


Menganalisis bagian validasi dari dataset COCO untuk diunduh di FiftyOne Dataset Zoo. Code di bawah digunakan untuk mengunduh dan memuat bagian validasi ke FiftyOne.

In [3]:
import fiftyone as fo
import fiftyone.zoo as foz

dataset = foz.load_zoo_dataset(
    "coco-2017",
    split="validation",
    dataset_name="evaluate-detections-tutorial",
)
dataset.persistent = True

INFO:fiftyone.zoo.datasets:Downloading split 'validation' to '/root/fiftyone/coco-2017/validation' if necessary


Found annotations at '/root/fiftyone/coco-2017/raw/instances_val2017.json'


INFO:fiftyone.utils.coco:Found annotations at '/root/fiftyone/coco-2017/raw/instances_val2017.json'


Images already downloaded


INFO:fiftyone.utils.coco:Images already downloaded


Existing download of split 'validation' is sufficient


INFO:fiftyone.zoo.datasets:Existing download of split 'validation' is sufficient


Loading existing dataset 'evaluate-detections-tutorial'. To reload from disk, either delete the existing dataset or provide a custom `dataset_name` to use


INFO:fiftyone.zoo.datasets:Loading existing dataset 'evaluate-detections-tutorial'. To reload from disk, either delete the existing dataset or provide a custom `dataset_name` to use


Lihat dataset untuk mengetahui yang sudah diunduh.

In [4]:
# Print some information about the dataset
print(dataset)

Name:        evaluate-detections-tutorial
Media type:  image
Num samples: 5000
Persistent:  True
Tags:        []
Sample fields:
    id:           fiftyone.core.fields.ObjectIdField
    filepath:     fiftyone.core.fields.StringField
    tags:         fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:     fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.ImageMetadata)
    ground_truth: fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)
    faster_rcnn:  fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)
    eval_tp:      fiftyone.core.fields.IntField
    eval_fp:      fiftyone.core.fields.IntField
    eval_fn:      fiftyone.core.fields.IntField


In [5]:
# Print a ground truth detection
sample = dataset.first()
print(sample.ground_truth.detections[0])

<Detection: {
    'id': '651037eeacf7381d2a0b0951',
    'attributes': {},
    'tags': [],
    'label': 'potted plant',
    'bounding_box': [
        0.37028125,
        0.3345305164319249,
        0.038593749999999996,
        0.16314553990610328,
    ],
    'mask': None,
    'confidence': None,
    'index': None,
    'supercategory': 'furniture',
    'iscrowd': 0,
}>


Catat bahwa deteksi kebenaran berada dalam kolom ground_truth pada sampel.

Sebelum berlanjut, aktifkan Aplikasi FiftyOne dan gunakan tampilan grafis untuk memeriksa dataset secara visual.

In [7]:
session = fo.launch_app(dataset)

Menambahkan hasil prediksi ke dalam dataset dan buat sejumlah prediksi untuk dianalisis. Code di bawah untuk melakukan proses inferensi menggunakan model Faster R-CNN pada 100 sampel acak dari dataset dan menyimpan hasil prediksinya pada kolom faster_rcnn dari sampel tersebut.

In [8]:
# Choose a random subset of 100 samples to add predictions to
predictions_view = dataset.take(100, seed=51)

In [9]:
from PIL import Image
from torchvision.transforms import functional as func

import fiftyone as fo

# Get class list
classes = dataset.default_classes

# Add predictions to samples
with fo.ProgressBar() as pb:
    for sample in pb(predictions_view):
        # Load image
        image = Image.open(sample.filepath)
        image = func.to_tensor(image).to(device)
        c, h, w = image.shape

        # Perform inference
        preds = model([image])[0]
        labels = preds["labels"].cpu().detach().numpy()
        scores = preds["scores"].cpu().detach().numpy()
        boxes = preds["boxes"].cpu().detach().numpy()

        # Convert detections to FiftyOne format
        detections = []
        for label, score, box in zip(labels, scores, boxes):
            # Convert to [top-left-x, top-left-y, width, height]
            # in relative coordinates in [0, 1] x [0, 1]
            x1, y1, x2, y2 = box
            rel_box = [x1 / w, y1 / h, (x2 - x1) / w, (y2 - y1) / h]

            detections.append(
                fo.Detection(
                    label=classes[label],
                    bounding_box=rel_box,
                    confidence=score
                )
            )

        # Save predictions to dataset
        sample["faster_rcnn"] = fo.Detections(detections=detections)
        sample.save()

print("Finished adding predictions")

 100% |█████████████████| 100/100 [10.0m elapsed, 0s remaining, 0.2 samples/s]    


INFO:eta.core.utils: 100% |█████████████████| 100/100 [10.0m elapsed, 0s remaining, 0.2 samples/s]    


Finished adding predictions


Menampilkan predictions_view di Aplikasi untuk melihat hasil prediksi yang telah ditambahkan.

In [10]:
session.view = predictions_view

**Mengevaluasi deteksi**
<br>Tinjau lebih lanjut prediksi yang telah disertakan ke dalam dataset.

**Melihat kotak pembatas**
<br>Mengakses dataset lengkap melalui Aplikasi:

In [11]:
# Resets the session; the entire dataset will now be shown
session.view = None

Hanya 100 sampel dalam predictions_view yang memiliki hasil prediksi di kolom faster_rcnn, sehingga beberapa sampel yang ditampilkan di atas tidak mempunyai kotak prediksi.

Jika ingin mengembalikan tampilan prediksi, bisa melakukannya melalui kode dengan session.view = predictions_view, atau juga bisa memanfaatkan bilah tampilan di Aplikasi untuk tujuan yang sama.

In [12]:
# Use the view bar to create an `Exists(faster_rcnn, True)` stage
# Now your view contains only the 100 samples with predictions in `faster_rcnn` field
session.show()

In [13]:
session.view = predictions_view

Masing-masing kolom dari sampel ditampilkan sebagai kotak centang di bilah samping kiri untuk memilih apakah ingin menampilkan deteksi kebenaran dasar atau hasil prediksi pada gambar.

Bisa klik sebuah gambar untuk memeriksa sampel tersebut lebih mendalam.

In [14]:
session.show()

**Memilih sampel tertentu**
<br>Memilih gambar dalam Aplikasi dengan menekan kotak centang saat gambar diarahkan. Setelah itu, dapat membentuk tampilan yang hanya berisikan sampel-sampel yang dipilih dengan mengeklik tanda centang berwarna oranye yang menunjukkan jumlah sampel yang dipilih di pojok kiri atas dan menekan *Only show selected samples*.

In [15]:
# Click the down caret on the `faster_rcnn` field of Fields Sidebar
# and apply a confidence threshold
session.show()

**Penentuan *Thresholding* di Python**
<br>FiftyOne menawarkan fitur untuk menulis pernyataan yang dapat mencocokkan, menyortir, dan memfilter deteksi berdasarkan sifat-sifatnya. Untuk informasi lebih lanjut, lihat bagian DatasetViews.
<br>Misalnya, bisa membuat sebuah tampilan secara otomatis yang hanya menampilkan deteksi dengan tingkat kepercayaan minimal 0,75 seperti di bawah:
<br>Parameter only_matches=False. Saat menyaring label, biasanya sampel yang tak lagi memiliki label akan dihilangkan dari tampilan. Tetapi, saat evaluasi, tidak menginginkan hal tersebut karena bisa mempengaruhi perbandingan hasil antar tampilan. Dengan mengatur only_matches=False, memastikan bahwa semua sampel tetap ada, meskipun ada yang tak memiliki label.

In [16]:
from fiftyone import ViewField as F

# Only contains detections with confidence >= 0.75
high_conf_view = predictions_view.filter_labels("faster_rcnn", F("confidence") > 0.75, only_matches=False)

In [17]:
# Print some information about the view
print(high_conf_view)

Dataset:     evaluate-detections-tutorial
Media type:  image
Num samples: 100
Sample fields:
    id:           fiftyone.core.fields.ObjectIdField
    filepath:     fiftyone.core.fields.StringField
    tags:         fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:     fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.ImageMetadata)
    ground_truth: fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)
    faster_rcnn:  fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)
    eval_tp:      fiftyone.core.fields.IntField
    eval_fp:      fiftyone.core.fields.IntField
    eval_fn:      fiftyone.core.fields.IntField
View stages:
    1. Take(size=100, seed=51)
    2. FilterLabels(field='faster_rcnn', filter={'$gt': ['$$this.confidence', 0.75]}, only_matches=False, trajectories=False)


In [18]:
# Print a prediction from the view to verify that its confidence is > 0.75
sample = high_conf_view.first()
print(sample.faster_rcnn.detections[0])

<Detection: {
    'id': '65105565dcc6f0478f78ee5d',
    'attributes': {},
    'tags': [],
    'label': 'person',
    'bounding_box': [
        0.2085207223892212,
        0.4248880043189396,
        0.20568811893463135,
        0.2677846613289422,
    ],
    'mask': None,
    'confidence': 0.9985470175743103,
    'index': None,
}>


In [19]:
# Load high confidence view in the App
session.view = high_conf_view

In [20]:
session.view = predictions_view

**Melakukan evaluasi**
<br>Melakukan evaluasi atas sampel dengan fungsi evaluate_detections(). Harus diperhatikan bahwa fungsi ini dapat ditemukan di kelas Dataset dan DatasetView, yang artinya bisa mengaplikasikan evaluasi pada high_conf_view untuk mengukur seberapa baik prediksi dengan tingkat *confidence* dalam dataset.
<br>Secara standar, fungsi ini akan mengadopsi protokol evaluasi COCO dan beberapa kelebihan lain yang bisa dimanfaatkan kemudian.

In [21]:
# Evaluate the predictions in the `faster_rcnn` field of our `high_conf_view`
# with respect to the objects in the `ground_truth` field
results = high_conf_view.evaluate_detections(
    "faster_rcnn",
    gt_field="ground_truth",
    eval_key="eval",
    compute_mAP=True,
)

Evaluating detections...


INFO:fiftyone.utils.eval.detection:Evaluating detections...


 100% |█████████████████| 100/100 [3.9s elapsed, 0s remaining, 26.6 samples/s]      


INFO:eta.core.utils: 100% |█████████████████| 100/100 [3.9s elapsed, 0s remaining, 26.6 samples/s]      


Performing IoU sweep...


INFO:fiftyone.utils.eval.coco:Performing IoU sweep...


 100% |█████████████████| 100/100 [1.4s elapsed, 0s remaining, 63.0 samples/s]         


INFO:eta.core.utils: 100% |█████████████████| 100/100 [1.4s elapsed, 0s remaining, 63.0 samples/s]         


**Mengkompilasi hasil**
<br>Objek yang dihasilkan dari prosedur evaluasi memberikan banyak cara yang memudahkan untuk menganalisis prediksi.
<br>Misalnya, menampilkan sebuah laporan klasifikasi dari 10 kelas terbanyak dalam dataset:

In [22]:
# Get the 10 most common classes in the dataset
counts = dataset.count_values("ground_truth.detections.label")
classes_top10 = sorted(counts, key=counts.get, reverse=True)[:10]

# Print a classification report for the top-10 classes
results.print_report(classes=classes_top10)

               precision    recall  f1-score   support

       person       0.89      0.75      0.82       228
          car       0.71      0.68      0.69        25
        chair       0.78      0.53      0.63        40
         book       0.63      0.23      0.33        53
       bottle       0.50      0.67      0.57        27
          cup       0.77      0.59      0.67        29
 dining table       0.50      0.33      0.40        21
traffic light       0.00      0.00      0.00         1
         bowl       0.80      0.24      0.36        17
      handbag       0.60      0.21      0.32        14

    micro avg       0.78      0.60      0.67       455
    macro avg       0.62      0.42      0.48       455
 weighted avg       0.78      0.60      0.66       455



Mengkalkulasi mean average-precision (mAP) dari alat deteksi:

In [23]:
print(results.mAP())

0.41052451745271507


<br>Mengingat evaluate_detections() menerapkan protokol evaluasi COCO yang resmi, nilai mAP ini akan sejalan dengan hasil yang diberikan oleh pycocotools.
<br>Selain itu, bisa menampilkan kurva precision-recall (PR) untuk beberapa kelas spesifik dalam model:

In [24]:
plot = results.plot_pr_curves(classes=["person", "car"])
plot.show()

**Analisis per sampel**
<br>Prosedur evaluasi menambahkan beberapa kolom baru pada dataset yang berisi data penting untuk membantu menilai prediksi di tingkat sampel.
<br>Dalam detail, setiap sampel sekarang memiliki kolom-kolom berikut:
<br>- eval_tp: menghitung prediksi yang True Positive (TP) pada sampel tersebut.
<br>- eval_fp: menghitung prediksi yang False Positive (FP) pada sampel tersebut.
<br>- eval_fn: menghitung prediksi yang False Negative (FN) pada sampel tersebut.

In [25]:
# Our dataset's schema now contains `eval_*` fields
print(dataset)

Name:        evaluate-detections-tutorial
Media type:  image
Num samples: 5000
Persistent:  True
Tags:        []
Sample fields:
    id:           fiftyone.core.fields.ObjectIdField
    filepath:     fiftyone.core.fields.StringField
    tags:         fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:     fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.ImageMetadata)
    ground_truth: fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)
    faster_rcnn:  fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)
    eval_tp:      fiftyone.core.fields.IntField
    eval_fp:      fiftyone.core.fields.IntField
    eval_fn:      fiftyone.core.fields.IntField


In [26]:
# Our detections have helpful evaluation data on them
sample = high_conf_view.first()
print(sample.faster_rcnn.detections[0])

<Detection: {
    'id': '65105565dcc6f0478f78ee5d',
    'attributes': {},
    'tags': [],
    'label': 'person',
    'bounding_box': [
        0.2085207223892212,
        0.4248880043189396,
        0.20568811893463135,
        0.2677846613289422,
    ],
    'mask': None,
    'confidence': 0.9985470175743103,
    'index': None,
    'eval_iou': 0.8703237658555171,
    'eval_id': '651037f6acf7381d2a0b32f9',
    'eval': 'tp',
}>


Kolom-kolom tambahan ini ada karena menggunakan parameter eval_key saat memanggil evaluate_detections(). Jika tidak menyertakan parameter ini, maka tidak ada data tambahan yang akan tercatat di sampel data.

In [27]:
print(dataset.list_evaluations())

['eval']


In [28]:
print(dataset.get_evaluation_info("eval"))

{
    "key": "eval",
    "version": "0.22.0",
    "timestamp": "2023-09-24T15:38:51.163000",
    "config": {
        "method": "coco",
        "cls": "fiftyone.utils.eval.coco.COCOEvaluationConfig",
        "pred_field": "faster_rcnn",
        "gt_field": "ground_truth",
        "iou": 0.5,
        "classwise": true,
        "iscrowd": "iscrowd",
        "use_masks": false,
        "use_boxes": false,
        "tolerance": null,
        "compute_mAP": true,
        "iou_threshs": [
            0.5,
            0.55,
            0.6,
            0.65,
            0.7,
            0.75,
            0.8,
            0.85,
            0.9,
            0.95
        ],
        "max_preds": 100,
        "error_level": 1
    }
}


In [29]:
# Load the view on which we ran the `eval` evaluation
eval_view = dataset.load_evaluation_view("eval")
print(eval_view)

Dataset:     evaluate-detections-tutorial
Media type:  image
Num samples: 100
Sample fields:
    id:           fiftyone.core.fields.ObjectIdField
    filepath:     fiftyone.core.fields.StringField
    tags:         fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:     fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.ImageMetadata)
    ground_truth: fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)
    faster_rcnn:  fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)
    eval_tp:      fiftyone.core.fields.IntField
    eval_fp:      fiftyone.core.fields.IntField
    eval_fn:      fiftyone.core.fields.IntField
View stages:
    1. Take(size=100, seed=51)
    2. FilterLabels(field='faster_rcnn', filter={'$gt': ['$$this.confidence', 0.75]}, only_matches=False, trajectories=False)


**Evaluation views**
<br>Setelah mendapatkan pemahaman tentang kinerja umum dari model, lakukan eksplorasi analisis per sampel dengan membuat sebuah tampilan untuk evaluasi.
<br>Setiap evaluasi yang telah disimpan di dataset dapat dijadikan sebagai dasar untuk membentuk sebuah tampilan evaluasi, dalam hal ini adalah sebuah tampilan per potongan yang membuat sampel untuk setiap prediksi TP, FP, dan FN dalam dataset. Dengan tampilan ini, dapat memudahkan filter dan mengurutkan deteksi berdasarkan kategori mereka (TP/FP/FN), nilai IoU (IoU, singkatan dari "Intersection over Union", merupakan metrik dalam deteksi objek yang mengukur kesesuaian antara bounding box prediksi dan kebenaran. Dihitung dari area perpotongan kedua bounding box dibagi dengan area gabungannya, dengan rentang nilai antara 0 hingga 1. 0 menandakan tidak ada perpotongan antara dua bounding box dan 1 menandakan kesempurnaan, di mana bounding box prediksi dan kebenaran dasar benar-benar bertepatan.) yang telah dievaluasi dan disesuaikan dengan objek dalam kerumunan.
<br>Tampilan evaluasi ini bisa diinisiasi baik melalui code Python ataupun langsung melalui Aplikasi, seperti yang diilustrasikan di bawah ini.

In [30]:
eval_patches = dataset.to_evaluation_patches("eval")
print(eval_patches)

Dataset:     evaluate-detections-tutorial
Media type:  image
Num patches: 40110
Patch fields:
    id:           fiftyone.core.fields.ObjectIdField
    sample_id:    fiftyone.core.fields.ObjectIdField
    filepath:     fiftyone.core.fields.StringField
    tags:         fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:     fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.ImageMetadata)
    ground_truth: fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)
    faster_rcnn:  fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)
    crowd:        fiftyone.core.fields.BooleanField
    type:         fiftyone.core.fields.StringField
    iou:          fiftyone.core.fields.FloatField
View stages:
    1. ToEvaluationPatches(eval_key='eval', config=None)


In [31]:
session.view = high_conf_view

**Lihat sampel dengan kinerja terbaik**
<br>Untuk menganalisis lebih lanjut, buat tampilan yang diurutkan berdasarkan eval_tp sehingga dapat melihat kasus-kasus dengan kinerja terbaik dari model (yaitu, sampel dengan TP yang paling banyak).

In [32]:
# Show samples with most true positives
session.view = high_conf_view.sort_by("eval_tp", reverse=True)

**Lihat sampel dengan kinerja terburuk**
<br>Dengan cara yang serupa, bisa mengurutkan berdasarkan kolom eval_fp untuk melihat kasus-kasus dengan kinerja terburuk dari model (yaitu, sampel dengan prediksi FP yang paling banyak).

In [33]:
# Show samples with most false positives
session.view = high_conf_view.sort_by("eval_fp", reverse=True)

In [34]:
# Compute metadata so we can reference image height/width in our view
dataset.compute_metadata()

In [35]:
#
# Create an expression that will match objects whose bounding boxes have
# area less than 32^2 pixels
#
# Bounding box format is [top-left-x, top-left-y, width, height]
# with relative coordinates in [0, 1], so we multiply by image
# dimensions to get pixel area
#
bbox_area = (
    F("$metadata.width") * F("bounding_box")[2] *
    F("$metadata.height") * F("bounding_box")[3]
)
small_boxes = bbox_area < 32 ** 2

# Create a view that contains only small (and high confidence) predictions
small_boxes_view = high_conf_view.filter_labels("faster_rcnn", small_boxes)

session.view = small_boxes_view

In [36]:
# Create a view that contains only small GT and predicted boxes
small_boxes_eval_view = (
    high_conf_view
    .filter_labels("ground_truth", small_boxes, only_matches=False)
    .filter_labels("faster_rcnn", small_boxes, only_matches=False)
)

# Run evaluation
small_boxes_results = small_boxes_eval_view.evaluate_detections(
    "faster_rcnn",
    gt_field="ground_truth",
)

Evaluating detections...


INFO:fiftyone.utils.eval.detection:Evaluating detections...


 100% |█████████████████| 100/100 [288.3ms elapsed, 0s remaining, 346.9 samples/s]      


INFO:eta.core.utils: 100% |█████████████████| 100/100 [288.3ms elapsed, 0s remaining, 346.9 samples/s]      


In [37]:
# Get the 10 most common small object classes
small_counts = small_boxes_eval_view.count_values("ground_truth.detections.label")
classes_top10_small = sorted(small_counts, key=counts.get, reverse=True)[:10]

# Print a classification report for the top-10 small object classes
small_boxes_results.print_report(classes=classes_top10_small)

               precision    recall  f1-score   support

       person       0.52      0.28      0.36        47
          car       0.40      0.33      0.36         6
        chair       1.00      0.14      0.25         7
         book       0.20      0.06      0.10        32
       bottle       0.50      0.46      0.48        13
          cup       0.75      0.56      0.64        16
 dining table       0.00      0.00      0.00         1
traffic light       0.00      0.00      0.00         1
         bowl       1.00      0.43      0.60         7
      handbag       0.00      0.00      0.00         3

    micro avg       0.52      0.27      0.36       133
    macro avg       0.44      0.23      0.28       133
 weighted avg       0.49      0.27      0.34       133



Format data COCO, terdiri dari anotasi kebenaran yang memiliki atribut iscrowd = 0 atau 1 yang menunjukkan apakah sebuah kotak mengandung beberapa contoh dari objek yang sama.

In [38]:
# View the `iscrowd` attribute on a ground truth object
sample = dataset.first()
print(sample.ground_truth.detections[0])

<Detection: {
    'id': '651037eeacf7381d2a0b0951',
    'attributes': {},
    'tags': [],
    'label': 'potted plant',
    'bounding_box': [
        0.37028125,
        0.3345305164319249,
        0.038593749999999996,
        0.16314553990610328,
    ],
    'mask': None,
    'confidence': None,
    'index': None,
    'supercategory': 'furniture',
    'iscrowd': 0,
}>


In [39]:
# Create a view that contains only samples for which at least one detection has
# its iscrowd attribute set to 1
crowded_images_view = high_conf_view.match(
    F("ground_truth.detections").filter(F("iscrowd") == 1).length() > 0
)

session.view = crowded_images_view

In [40]:
session.view = crowded_images_view.sort_by("eval_fp", reverse=True)

In [41]:
session.view = high_conf_view.sort_by("eval_fp", reverse=True)

In [42]:
session.show()

In [43]:
session.freeze()  # screenshot the active App for sharing

In [44]:
# Tag all highly confident false positives as "possibly-missing"
(
    high_conf_view
        .filter_labels("faster_rcnn", F("eval") == "fp")
        .select_fields("faster_rcnn")
        .tag_labels("possibly-missing")
)

In [45]:
# Export all labels with the `possibly-missing` tag in CVAT format
(
    dataset
        .select_labels(tags=["possibly-missing"])
        .export("/path/for/export", fo.types.CVATImageDataset)
)

Directory '/path/for/export' already exists; export will be merged with existing files


Found multiple fields ['ground_truth', 'faster_rcnn'] with compatible type <class 'fiftyone.core.labels.Detections'>; exporting 'ground_truth'


INFO:fiftyone.core.collections:Found multiple fields ['ground_truth', 'faster_rcnn'] with compatible type <class 'fiftyone.core.labels.Detections'>; exporting 'ground_truth'


 100% |███████████████████| 68/68 [278.7ms elapsed, 0s remaining, 244.0 samples/s]      


INFO:eta.core.utils: 100% |███████████████████| 68/68 [278.7ms elapsed, 0s remaining, 244.0 samples/s]      


Referensi:
- https://docs.voxel51.com/tutorials/evaluate_detections.html
- https://cocodataset.org/#home